# 빅분기 작업형2 Baseline
- 결과를 보려면 print() 함수 필요
- 일반적으로 train, test 두 파일이 제공 $\rightarrow$ train에서 X, y 분리해야함
    - 만약 X_train, X_test, y_train 세 파일이 제공됐다면 위 처럼 X, y를 분리하지 않아도 될 것

## 0. 데이터 로드

In [ ]:
import pandas as pd
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")


print(train.shape)
print(train.head(), "\n")
print(test.shape, "\n")
print(test.head())

# 1. EDA

### 1-1) train EDA

In [ ]:
print(train.info(), "\n")
print(train.isnull.sum(), "\n") # 결측치 확인
print(train.describe(), "\n")

### 1-2) test EDA

In [ ]:
print(test.info(), "\n")
print(test.isnull.sum(), "\n") # 결측치 확인
print(test.describe(), "\n")

In [ ]:
print(train.describe(include="O"), "\n") # train 범주형 변수 unique 확인
print(test.describe(include="O"), "\n") # test 범주형 변수 unique 확인

- train, test에서 범주형 변수의 nuniqie 값 확인하지만.. 아마 시험에서는 값 다르지 않을 것임
- 다르게 되면 처리해야할 게 늘어남 

## 2. 데이터 전처리

### 2-1) 결측치 처리 & ID같은 불필요 칼럼 제거 & train에서 X, y 분리
- pop 함수는 axis 따로 지정안해도 되는데 drop은 해줘야함 

In [ ]:
# 결측치는 평균보다 이상값에 덜 민감한 중앙값으로 대치
# 데이터에 따라 결측치 방법 고려해보기 (0으로 대치, 평균 대치 ...)
train["칼럼"] = train["칼럼"].fillna(train["칼럼"].median())
test["칼럼"] = test["칼럼"].fillna(test["칼럼"].median())

# 결측치 채워졌나 확인
print(train.isnull().sum(), "\n")
print(test.isnull().sum(), "\n")

# 예측에 불필요한 ID 칼럼 제거
train = train.drop("ID", axis=1)
print(train.head(1), "\n")

# test 데이터 ID 분리
test_ID = test.pop('ID')
print(test.head(1), "\n")

y = train.pop("타켓 칼럼") # X, y  분리
print(y.head(), "\n")

### 2-2) 스케일링
- train에만 fit_transform( ) 해줘야함
- test는 transform( )

In [ ]:
print(train.info(), "\n")

con_cols = train.select_dtypes(exclude="object").copy().columns # 수치형 칼럼 선택
print("수치형 칼럼 : ", con_cols, "\n")

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train[con_cols] = scaler.fit_transform(train[con_cols])
train[con_cols] = scaler.transform(train[con_cols])

### 2-3) 인코딩
- 어차피 모델은 랜포, lgbm 비교할 거니까 라벨인코딩, 원핫인코딩 두 방법 다 가능할텐데 범주형 변수 내 nunique 값이 많으면 라벨인코딩 하자
- train, test nunique 값이 다르다면? 
    - train, test를 `pd.concat()`으로 합친 후 인코딩해서, 다시 분리

In [ ]:
# 원핫인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# Train 데이터와 Test 데이터의 열 순서를 비교하여 동일한 순서로 정렬
test = test[train.columns]

#### 원핫인코딩 후 왜 train, test를 동일한 열 순서로 해야할까? : 올바른 예측, 정확도를 보장하기 위해 !
- Train 데이터와 Test 데이터의 열 순서가 일치하지 않으면 모델이 올바르게 예측을 수행할 수 없음
- 모델이 학습한 피처 순서와 테스트할 때 입력하는 데이터의 피처 순서가 일치해야함
    - 만약 Train 데이터와 Test 데이터의 열 순서가 일치하지 않을 경우, 모델은 학습한 피처의 순서와 다른 피처 순서를 입력으로 받게 되므로 잘못된 결과를 예측할 수 있음
- 모델은 Train 데이터를 학습할 때 피처들의 순서에 의존하여 가중치를 조정하고 학습
    - 따라서, Train 데이터와 Test 데이터에 동일한 열 순서를 갖도록 정렬해야 모델이 정확한 예측을 수행할 수 있음

In [ ]:
# 라벨인코딩
cat_cols = train.select_dtypes(include="object").copy().columns # 범주형 칼럼 선택

from sklearn.preprocessing import LabelEncoder
for col in cat_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

print(train.head(), "\n")

In [ ]:
# 데이터 합치기
df = pd.concat([train, test])

# 인코딩 진행
from sklearn.preprocessing import LabelEncoder

cat_cols = df.select_dtypes(include="object").copy().columns

for col in cat_cols :
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    
# 인코딩 진행 후 다시 분리
train = df[:train.shape[0]].copy()
test = df[train.shape[0]:].copy()

## 3. 검증 데이터 분리
- 분류일 때만 stratify=y 
    - 데이터를 분할할 때 클래스의 분포를 유지

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, y, random_state=1, stratify=y)

print(X_train.shape, X_val.shape, "\n")

## 4. 모델링
- 랜덤포레스트, lgbm 두 개 준비

### 4-1) 성능 지표 : 함수 안 매개변수 순서는 실제값, 예측값 순 !
- 아래 준비한 성능지표 외에 모르거나 헷갈리는게 나오면 help(), dir() 활용하자
- 기본적으로 model.score(x_val, y_val) 하면 분류모델은 정확도를, 회귀모델은 결정계수를 출력한다.

#### 분류 : roc_auc_score, f1_score (다중분류 macro f1), accuracy_score

In [ ]:
# 분류
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, classification_report

#### 회귀 : RMSE, MSE, MAE, R2

In [ ]:
# 회귀
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

### 4-2) 분류

`model.predict_proba(x_val)`을 하면 클래스에 따라 나올 확률 값을 행렬로 나타내줌.  
즉 칼럼의 순서가 각각의 클래스가 나올 확률 이므로 제출할 형태가 '1일 확률' 이라면 `model.predict_proba(X_val)[:, 1]`을 선택



- 예측 확률은 이진분류라고 할 때 0(여자), 1(남자)일 확률 이렇게 두 값이 나옴
- 따라서 남자일 확률은 rf.predict_proba(X_val)[:, 1] 이런식으로 지정해줘야함

```python
rf_pred = rf.predict_proba(X_val) # val 셋에서 남자일 예측 '확률'
print(rf_pred)

[[0.64 0.36]
 [0.63 0.37]
 [0.68 0.32]
 ...
 [0.8  0.2 ]
 [0.55 0.45]
 [0.69 0.31]]
 ```

#### 랜덤포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=1)
rf.fit(X_train, y_train)

rf_pred = rf.predict(X_val) # 예측 클래스
rf_pred_proba = rf.predict_proba(X_val)[:, 1] # 클래스 1에 대한 예측 확률 

# roc_auc_score : 이진분류일 때. 다중분류일 땐  average='macro' 옵션 추가 가능 ... 
# roc_auc_score만 확률값이랑 비교
rf_roc = roc_auc_score(y_val, rf_pred_proba)

# macro-f1 : 실제값, 예측값 순
rf_f1 =  f1_score(y_val, rf_pred, average = "macro") # 다중분류일 때 average = "macro" 옵션 사용 !
print(rf_f1)

# accuracy_score
rf_acc = accuracy_score(y_val, rf_pred)

#### lgbm

In [ ]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(random_state=1)
lgbm.fit(X_train, y_train)

lgbm_pred = lgbm.predict(X_val)
lgbm_pred_proba = lgbm.predict_proba(X_val)[:, 1]

# roc_auc_score
lgbm_roc = roc_auc_score(y_val, lgbm_pred_proba)

# f1_score
lgbm_f1 = f1_score(y_val, lgbm_pred, average="macro") # macro는 다중분류일 때

# accuracy_score
lgbm_acc = accuracy_score(y_val, lgbm_pred)

- roc_auc_score만 'proba'랑 비교 !!

### 4-3) 회귀

#### 랜덤포레스트
RandomForestRegressor() 매개변수 디폴트 값
```
   n_estimators=100,
    criterion='mse',
    max_depth=None,
```

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=1)
rf.fit(X_train, y_train)

rf_pred = rf.predict(X_val) # 검증용 X_val

# rmse
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_pred))

# r2_score
rf_r2 = r2_score(y_val, rf_pred)

#### lgbm
LGBMRegressor() 매개변수 디폴트 값
```
    max_depth=-1,
    learning_rate=0.1,
    n_estimators=100,
```

In [3]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(random_state=1)
lgbm.fit(X_train, y_train)

lgbm_pred = lgbm.predict(X_val) # 검증용 X_val

# rmse
lgbm_rmse = np.sqrt(mean_squared_error(y_val, lgbm.pred))

# r2_score
lgbm_r2 = r2_score(y_val, lgbm_pred)

### 최종 모델 선택해서 test 데이터에 대해 pred / pred_proba 지정해주기 !

In [ ]:
# 분류
pred = rf.predict(test) # 예측 클래스
pred_proba = rf.predict_proba(test)[:, 1] # 클래스 1에 대한 예측 확률 

pred = lgbm.predict(test)
pred_proba = lgbm.predict_proba(test)[:, 1]

# 회귀
pred = rf.predict(test)
pred = lgbm.predict(test)

## 5) 제출 : df & csv 저장

In [ ]:
# 예측 결과 -> 데이터 프레임

submit = pd.DataFrame({
    'ID': test_ID,
    'Segmentation': pred
})
submit.to_csv("submission.csv", index=False)

In [ ]:
check = pd.read_csv("submission.csv") # 결과 확인
print(check.head())